# Importación de módulos

In [10]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Importación de datos

In [11]:
# Copia los datos de un documento CSV en un DataFrame.
data_df = pd.read_csv('synchronous_machine.csv')
data_df

,Iy,PF,e,dIf,If
0,3.0,0.66,0.34,0.383,1.563
1,3.0,0.68,0.32,0.372,1.552
2,3.0,0.70,0.30,0.360,1.540
3,3.0,0.72,0.28,0.338,1.518
4,3.0,0.74,0.26,0.317,1.497
...,...,...,...,...,...
552,6.0,0.91,0.09,0.142,1.322
553,6.0,0.93,0.07,0.151,1.331
554,6.0,0.95,0.05,0.160,1.340
555,6.0,0.97,0.03,0.160,1.340


# Análisis de los datos

In [12]:
'''/* 
Recopila y muestra la información necesaria para conocer si hay outliers.
*/'''

# Retorna media, desviación estándar, min, cuantiles 25,50,75% y max.
display(data_df.describe())
# Retorna si hay valores en blanco o no numericos.
display(data_df.isna().sum())

# Establece los datos necesarios y títulos de eje para mostrar varias gráficas de caja.
figIy = px.box(data_df, y='Iy')
figIy.show()
figPF = px.box(data_df, y='PF')
figPF.show()
fige = px.box(data_df, y='e')
fige.show()
figdIf = px.box(data_df, y='dIf')
figdIf.show()
figIf = px.box(data_df, y='If')
figIf.show()

'''/* 
Function: IQR_outlier

Confirma de manera numérica la existencia de outliers.

Parameters:

    data_df - Datos recopilados del CSV.
    
Returns:

    Un DataFrame que contiene si un dato se sale de los límites calculados.
    Si un valor es igual a NaN significa que no es un outliers.
*/'''

def IQR_outlier(data_df):
    
    # Guarda el resultado del cuartil del 25%
    q1 = data_df.quantile(0.25)
    # Guarda el resultado del cuartil del 75%
    q3 = data_df.quantile(0.75)
    
    # Calcula el rango entre cuantiles
    IQR = q3-q1 
    
    ''' 
    Revisa si hay datos afuera de cada límite.
    Los limites se calculan de la siguiente manera: 
        Límite superior: q3 + (1.5*IQR)
        Límite inferior: q1 - (1.5*IQR)
    '''
    
    outliers = data_df[((data_df<(q1-1.5*IQR)) | (data_df>(q3+1.5*IQR)))]

    return outliers

# Iprime el resultado de la función anterior.
print(IQR_outlier(data_df))

,Iy,PF,e,dIf,If
count,557.000000,557.000000,557.000000,557.000000,557.000000
mean,4.499820,0.825296,0.174704,0.350659,1.530659
std,0.896024,0.103925,0.103925,0.180566,0.180566
min,3.000000,0.650000,0.000000,0.037000,1.217000
25%,3.700000,0.740000,0.080000,0.189000,1.369000
50%,4.500000,0.820000,0.180000,0.345000,1.525000
75%,5.300000,0.920000,0.260000,0.486000,1.666000
max,6.000000,1.000000,0.350000,0.769000,1.949000


Iy     0
PF     0
e      0
dIf    0
If     0
dtype: int64

     Iy  PF   e  dIf  If
0   NaN NaN NaN  NaN NaN
1   NaN NaN NaN  NaN NaN
2   NaN NaN NaN  NaN NaN
3   NaN NaN NaN  NaN NaN
4   NaN NaN NaN  NaN NaN
..   ..  ..  ..  ...  ..
552 NaN NaN NaN  NaN NaN
553 NaN NaN NaN  NaN NaN
554 NaN NaN NaN  NaN NaN
555 NaN NaN NaN  NaN NaN
556 NaN NaN NaN  NaN NaN

[557 rows x 5 columns]


# Separación de los set de datos y normalizacón

In [13]:
'''/* 
Esta sección prepara los datos recopilados anteriormente en subsecciones para entrenamieto y prueba.
Además de esto, normaliza los datos para poder ser ingresados a una función sigmoide.
*/'''

# Arreglo que contiene los datos de prueba dados por el profresor.
tarea_data = [[3, 0.64, 0.01, 0.031, 0], [4.38, 0.78, 0.14, 0.402, 0], [6, 1.01, 0.35,0.799, 0]]
# Convierte un arreglo en DataFrame con las columnas llamadas: 'Iy', 'PF', 'e', 'dIf'.
tarea_data = pd.DataFrame(tarea_data, columns=['Iy', 'PF', 'e', 'dIf', 'If'])

# Guarda las características del conjunto de datos antes de normalizar.
train_stats = data_df.describe()
tarea_stats = tarea_data.describe()

# Se transpone el DataFrame para que las columnas y filas se inviertan entre ellas.
train_stats = train_stats.transpose()
tarea_stats = tarea_stats.transpose()

'''/* 
Function: norm

Normaliza los datos a través de una función min-max entre 0 y 1 para ser alimentados a una función sigmoide.

Parameters:

    x - Datos a normalizar.
    
Returns:

    Los datos alimentados ya normalizados.
*/'''

def norm(x, stats):
    return((x-stats['min'])/(stats['max']-stats['min'])) #min-max norm

'''/* 
Function: reverse_norm

Desnormaliza los datos a través de la funcion inversa de la min-max.

Parameters:

    x - Datos normalizados a revertir.
    
Returns:

    Los datos alimentados ya desnormalizados.
*/'''

def reverse_norm(x):
    return(x*(train_stats['max']-train_stats['min'])+train_stats['min'])

# Normaliza todo el conjunto de datos
data_df = norm(data_df, train_stats)
# Mueve 80% de los datos en un subconjuto de entrenamiento y los aleatoriza.
train_df = data_df.sample(frac=0.8, random_state=0)
# Mueve el restante de los datos en un subconjuto de prueba y los aleatoriza.
test_df = data_df.drop(train_df.index)
# Guarda los datos de la comlumna If en una variable para las etiquetas de entrenamiento.
train_labels = train_df.pop('If')
# Guarda los datos de la comlumna If en una variable para las etiquetas de prueba.
test_labels = test_df.pop('If')

# Se normalizan los datos dados por el profesor
tarea_data = norm(tarea_data, tarea_stats)
# Se elimina la columna de las etiquetas
tarea_df = tarea_data.drop(['If'], axis=1)


# Creación del modelo

In [14]:
'''/* 
Function: my_model

Genera el modelo de regresión MLP y lo compila. El modelo consiste de lo siguiente:
    - Una capa de entradas de forma train_df.keys() que es igual a la cantidad de características.
    - Una capa oculta con dos neuronas y como función de activación una función sigmoide.
    - Una capa oculta con dos neuronas y como función de activación una función sigmoide.
    - Una capa de salida con una neurona y como función de activación una función sigmoide.
    - Utiliza ADAM como optimizador.
    - La función de pérdida es la Suma Cuadrática del Error.
    
Parameters:

    my_learning_rate - Razón de aprendizaje.
    
Returns:

    El modelo neuronal ya creado.
*/'''

def my_model(my_learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2, input_shape=[len(train_df.keys())], activation='sigmoid'),
        tf.keras.layers.Dense(2, activation='sigmoid'),#se hace una hidden layer de 3 neuronas con activacion sigmoid
        tf.keras.layers.Dense(1, activation='sigmoid') #capa de output
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss="mean_squared_error", #funcion de perdida
                metrics=[tf.keras.metrics.MeanSquaredError()] #funcion de metricas para evaluar
                )
    
    return model

# Entrenamiento del modelo

In [15]:
'''/* 
Function: train_model

Alimenta los datos correspondientes a las características y etiquetas al modelo y conduce el proceso de entrenamiento y validación.
Además de esto, también le establece al modelo la cantidad de ciclos de entrenamiento y el tamaño del batch.
Por último crea un subconjunto de validación con un 25% de los datos de entrenamiento.
    
Parameters:

    model - El modelo creado anteriormente.
    features - Conjunto que contiene las características.
    labels - Conjunto que contiene las etiquetas.
    epochs - Cantidad de ciclos de entrenamiento
    batch_size - Tamaño del batch.
    
Returns:

    Los resultados del entrenamiento.
*/'''

def train_model(model, features, labels, epochs, batch_size):
    early_stop= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) #patience es la cant de epochs antes de chequear el improvement
    history = model.fit(
        x=features,
        y=labels,
        epochs=epochs,
        batch_size = batch_size,
        validation_split=0.25,
        callbacks = early_stop
    )
    
    # Guarda los resultados obtenidos del proceso de entrenamiento y validación en un DataFrame.
    # Estos resultados son las pérdidas y el número del ciclo correspondiente.
    hist= pd.DataFrame(history.history)
    # Añade al DataFrame la información sobre los ciclos.
    hist['epoch'] = history.epoch
    # Guarda los datos de error en una variable para uso futuro.
    mse = hist['mean_squared_error']
    
    return hist, mse

# Ploteo de gráficas

In [16]:
'''/* 
Function: plot_loss_curve

Grafica las curvas de pérdida correspondientes al entrenamiento y la validación.
    
Parameters:

    history - Resultados provenientes del proceso de entrenamiento.
    
Returns:

    Las gráficas ya creadas.
*/'''

def plot_loss_curve(history):
    hist = history
    # Cambia los títulos de cada columna que contiene los datos de pérdida por una versión más legible.  
    labels = {"mean_squared_error":"Training Loss", "val_mean_squared_error":"Validation Loss"}
    hist.rename(columns = labels, inplace = True)
    
    # Crea la figura, establece los títulos de eje y la paleta de colors
    fig = px.line(hist, x='epoch', y=['Training Loss', 'Validation Loss'],
                title='Gráficas de Pérdida de Entrenamiento y Evaluación',
                labels={"epoch": "Epoch", "value":"Mean Square Error", "variable":"Curvas de Pérdida"},
                color_discrete_map={
                "Training Loss": "#46039f", "Validation Loss": "#fb9f3a"})
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()

'''/* 
Function: plot_predictions

Grafica las etiquetas de prueba contra las predicciones hechas por la red neuronal.
    
Parameters:

    predictions - Datos por graficar.
    
Returns:

    La gráfica ya creada.
*/'''

def plot_predictions(predictions):
    
    # Se crea una traza de puntos.
    trace1 = go.Scatter(
        x = predictions['If'],
        y = predictions['If Predictions'],
        name = 'Predicciones',
        mode='markers',
    )
    
    # Se crea una traza de línea.
    trace2 = go.Line(
        x= predictions['If'],
        y = predictions['If'],
        name = 'Datos Reales',
        yaxis='y2'
    )
    
    # Se crea la figura 
    fig = make_subplots(specs=[[{"secondary_y": True}]], x_title='If', y_title='Predicción de If')
    # Se le agrega un título.
    fig.update_layout(title_text="Gráfica De Predicciones Contra Datos Reales")
    # Se le agrega la primera traza.
    fig.add_trace(trace1)
    # Se le agrega la segunda traza.
    fig.add_trace(trace2,secondary_y=False)
    fig.update_layout(template='plotly_white')
    
    fig.show()
    

# Hyperparámetros

In [17]:
learning_rate = 0.01
epochs = 100
batch_size = 8

# Llamado de funciones

In [18]:
# Llama a la función para crear el modelo y lo guarda.
model = my_model(learning_rate)
# Invoca a la función de entrenamiento y guarda los resultados.
history, mse = train_model(model, train_df, train_labels, epochs, batch_size)

Epoch 1/100
42/42 [==============================] - 1s 5ms/step - loss: 0.0596 - mean_squared_error: 0.0596 - val_loss: 0.0640 - val_mean_squared_error: 0.0640
Epoch 2/100
42/42 [==============================] - 0s 2ms/step - loss: 0.0555 - mean_squared_error: 0.0555 - val_loss: 0.0599 - val_mean_squared_error: 0.0599
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.0517 - mean_squared_error: 0.0517 - val_loss: 0.0546 - val_mean_squared_error: 0.0546
Epoch 4/100
42/42 [==============================] - 0s 2ms/step - loss: 0.0457 - mean_squared_error: 0.0457 - val_loss: 0.0457 - val_mean_squared_error: 0.0457
Epoch 5/100
42/42 [==============================] - 0s 2ms/step - loss: 0.0359 - mean_squared_error: 0.0359 - val_loss: 0.0335 - val_mean_squared_error: 0.0335
Epoch 6/100
42/42 [==============================] - 0s 2ms/step - loss: 0.0241 - mean_squared_error: 0.0241 - val_loss: 0.0212 - val_mean_squared_error: 0.0212
Epoch 7/100
42/42 [===============

In [19]:
# Llama a la función de las gráficas.
plot_loss_curve(history)
display(history)

,loss,Training Loss,val_loss,Validation Loss,epoch
0,0.059637,0.059637,0.063961,0.063961,0
1,0.055472,0.055472,0.059902,0.059902,1
2,0.051725,0.051725,0.054554,0.054554,2
3,0.045739,0.045739,0.045745,0.045745,3
4,0.035896,0.035896,0.033550,0.033550,4
...,...,...,...,...,...
95,0.000257,0.000257,0.000223,0.000223,95
96,0.000245,0.000245,0.000230,0.000230,96
97,0.000233,0.000233,0.000250,0.000250,97
98,0.000230,0.000230,0.000215,0.000215,98


# Predicciones

In [20]:
# Hace predicciones usando el conjunto de datos de prueba.
test_predictions = model.predict(test_df).flatten()
tarea_predictions = model.predict(tarea_df).flatten()

# Crea una arreglo de ceros con las dimensiones forma (111, 4) lo cuál es la cant de filas y de columnas -1 de los datos del CSV.
pivot1 = np.zeros((111,4))
# En la columna que falta se le agregan los labels de test normalizados.
pivot1 = np.insert(pivot1, 4, test_labels, axis =1)

# Crea un arreglo de ceros con las dimensiones de los datos del CSV.
pivot2 = np.zeros((111,5))

#El ciclo cambia los ceros en la última columna por los datos de la predicción.
for i in range(111):
    for j in range(5):
        pivot2[i,4] = test_predictions[i]

# Guarda las predicciones en un DataFrame.
test_predictions_df1 = pd.DataFrame(pivot1, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
test_predictions_df2 = pd.DataFrame(pivot2, columns=['Iy', 'PF', 'e', 'dIf', 'If'])

# Desnormaliza las predicciones y los labels.
test_predictions_df1 = reverse_norm(test_predictions_df1)
test_predictions_df2 = reverse_norm(test_predictions_df2)

# Renombra la columna "If" por "If Predictions" en el dataframe de las predicciones
test_predictions_df2.rename(columns = {'If':'If Predictions'}, inplace = True)
test_predictions_df = pd.concat([test_predictions_df1['If'], test_predictions_df2['If Predictions']], axis= 1)

# Llama a la función que grafica las predicciones sobre los datos.
plot_predictions(test_predictions_df)

# Calcula aparte el RMS de las predicciones
rms= mean_squared_error(test_predictions_df1['If'],test_predictions_df2['If Predictions'])
print(rms)

# Acomoda las predicciones en una matriz con cinco columnas, una para cada variable
pivot3 = np.zeros((len(tarea_predictions),5))
for i in range(len(tarea_predictions)):
        for j in range(5):
            pivot3[i,4] = tarea_predictions[i]

# Se transforma en DataFrame
pivot_df = pd.DataFrame(pivot3, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
pivot_df = reverse_norm(pivot_df)

1/1 [==============================] - 0s 21ms/step


c:\Users\Alienware\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




0.00010192571570989236


# Parte 3a

In [21]:
import itertools


# Define the values in each column
col1 = [3, 4.38, 6]
col2 = [0.64, 0.78, 1.01]
col3 = [0.01, 0.14, 0.35]
col4 = [0.031, 0.402, 0.799]

# Se generan todas las posibles combinaciones de resultados
X_n= list(itertools.product(col1, col2, col3, col4))

tarea_data = pd.DataFrame(X_n, columns=['Iy', 'PF', 'e', 'dIf'])
tarea_data[ 'If']=0
tarea_data_stats=tarea_data.describe().transpose()

# Normaliza los datos
tarea_data_norm = ((tarea_data-tarea_data_stats['min'])/(tarea_data_stats['max']-tarea_data_stats['min']))

tarea_data_norm = tarea_data_norm.drop(['If'], axis=1)
display(tarea_data_norm)

,Iy,PF,e,dIf,If
0,3.0,0.64,0.01,0.031,0
1,3.0,0.64,0.01,0.402,0
2,3.0,0.64,0.01,0.799,0
3,3.0,0.64,0.14,0.031,0
4,3.0,0.64,0.14,0.402,0
...,...,...,...,...,...
76,6.0,1.01,0.14,0.402,0
77,6.0,1.01,0.14,0.799,0
78,6.0,1.01,0.35,0.031,0
79,6.0,1.01,0.35,0.402,0


,count,mean,std,min,25%,50%,75%,max
Iy,81.0,4.460000,1.233690,3.000,3.000,4.380,6.000,6.000
PF,81.0,0.810000,0.153485,0.640,0.640,0.780,1.010,1.010
e,81.0,0.166667,0.140952,0.010,0.010,0.140,0.350,0.350
dIf,81.0,0.410667,0.315548,0.031,0.031,0.402,0.799,0.799
If,81.0,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000


,Iy,PF,e,dIf
0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.000000,0.483073
2,0.0,0.0,0.000000,1.000000
3,0.0,0.0,0.382353,0.000000
4,0.0,0.0,0.382353,0.483073
...,...,...,...,...
76,1.0,1.0,0.382353,0.483073
77,1.0,1.0,0.382353,1.000000
78,1.0,1.0,1.000000,0.000000
79,1.0,1.0,1.000000,0.483073


In [23]:

model=tf.keras.models.load_model('Modelos_regresion/m7')# Se carga el mejor modelo
tarea_predictions = model.predict(tarea_data_norm).flatten()#Se hacen las predicciones
pivot3 = np.zeros((len(tarea_predictions),5))
for i in range(len(tarea_predictions)):
        for j in range(5):
            pivot3[i,4] = tarea_predictions[i]
            
pivot_df = pd.DataFrame(pivot3, columns=['Iy', 'PF', 'e', 'dIf', 'If'])
pivot_df = pivot_df*(pivot_df.describe().transpose()['max']-pivot_df.describe().transpose()['min'])+pivot_df.describe().transpose()['min']


test_predictions_df = pd.concat([ tarea_data.drop(['If'], axis=1) , pivot_df['If']], axis= 1)
test_predictions_df.to_csv('partea.csv', index=False)
display(test_predictions_df)
display(pivot_df['If'].describe())

3/3 [==============================] - 0s 2ms/step


,Iy,PF,e,dIf,If
0,3.0,0.64,0.01,0.031,0.069713
1,3.0,0.64,0.01,0.402,0.786594
2,3.0,0.64,0.01,0.799,0.976368
3,3.0,0.64,0.14,0.031,0.056531
4,3.0,0.64,0.14,0.402,0.673992
...,...,...,...,...,...
76,6.0,1.01,0.14,0.402,0.392904
77,6.0,1.01,0.14,0.799,0.857531
78,6.0,1.01,0.35,0.031,0.047448
79,6.0,1.01,0.35,0.402,0.273007


count    81.000000
mean      0.498226
std       0.366889
min       0.043051
25%       0.058567
50%       0.500998
75%       0.857531
max       0.976584
Name: If, dtype: float64

# Ceteris Paribus

In [ ]:
# Se crean arreglos con una columna variable y con números del 2 al 4 para cambiar esos espacios por constantes.
# Los 5 son para cambiarlos por valores con ruido aleatorio de 5%, 15% o 25%

# Arreglo dónde la columna 0 es variable, corresponde a la variable Iy
arr1 = np.array([[0, 2, 3, 4], [0.46, 2, 3, 4], [1, 2, 3, 4], 
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4],
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4],
                [5, 2, 3, 4], [5, 2, 3, 4], [5, 2, 3, 4]])

# Arreglo dónde la columna 1 es variable, corresponde a la variable PF
arr2 = np.array([[2, 0, 3, 4], [2, 0.378378, 3, 4], [2, 1, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4],
                [2, 5, 3, 4], [2, 5, 3, 4], [2, 5, 3, 4]])

# Arreglo dónde la columna 2 es variable, corresponde a la variable e
arr3 = np.array([[2, 3, 0, 4], [2, 3, 0.382353, 4], [2, 3, 1, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4],
                [2, 3, 5, 4], [2, 3, 5, 4], [2, 3, 5, 4]])

# Arreglo dónde la columna 3 es variable, corresponde a la variable dIf
arr4 = np.array([[2, 3, 4, 0], [2, 3, 4, 0.483073], [2, 3, 4, 1],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5],
                [2, 3, 4, 5], [2, 3, 4, 5], [2, 3, 4, 5]])

# Reemplazan los 2 con un float aleatorio entre 0 y 1 
arr1[arr1 == 2] = np.random.random()
arr2[arr2 == 2] = np.random.random()
arr3[arr3 == 2] = np.random.random()
arr4[arr4 == 2] = np.random.random()

# Reemplazan los 3 con un float aleatorio entre 0 y 1
arr1[arr1 == 3] = np.random.random()
arr2[arr2 == 3] = np.random.random()
arr3[arr3 == 3] = np.random.random()
arr4[arr4 == 3] = np.random.random()

# Reemplazan los 4 con un float aleatorio entre 0 y 1
arr1[arr1 == 4] = np.random.random()
arr2[arr2 == 4] = np.random.random()
arr3[arr3 == 4] = np.random.random()
arr4[arr4 == 4] = np.random.random()


'''/* 
Function: complete

Cambia los 5 en los arreglos con cualquiera de los 3 primeros datos con un ruido aleatorio de 5%, 15% o 25%.
Manteniendo el rango entre 0 y 1.
    
Parameters:

    arr - Arreglo de datos.
    
Returns:

    El arreglo con 12 valores diferentes para la variable en evaluación.
*/'''

def complete(arr):
    porcentajes = [0.05, 0.15, 0.25]
    valores = [0,1,2]
    for i in range(len(arr)):
        for j in range(4):
            if ((arr[i][j] == 5)):
                rand1 = random.randint(0, 2)
                valor = valores[rand1]
                rand2 = random.randint(0, 2)
                porcentaje = porcentajes[rand2]
                if(valor != 2):
                    arr[i][j] = arr[valor][j] + arr[valor][j]*porcentaje
                else:
                    arr[i][j] = arr[valor][j] - arr[valor][j]*porcentaje
    return arr

# Guarda en arreglo creado por la función complete en otro con el nombre de la variable con la que se trabaja.
iy = complete(arr1)
pf = complete(arr2)
e = complete(arr3)
dif = complete(arr4)

# Predice los valores para cada uno de los arreglos
iy_pred = model.predict(iy)
pf_pred = model.predict(pf)
e_pred = model.predict(e)
dif_pred = model.predict(dif)

'''/* 
Function: cp_plot

Grafica las predicciones hechas contra a la variable en evaluación.
    
Parameters:

    data - Datos por graficar.
    var - Nombre de la variable en evaluación.
    
Returns:

    La gráfica ya creada.
*/'''

def cp_plot(data, var):
    # Crea la figura, establece los títulos de eje y los rangos de los ejes
    fig = px.line(data, x='x', y='y',
                labels={"x": "{}".format(var), "y":"Predicción"}, range_y=[0,1], range_x=[0,1])
    # Actualiza el tema de la gráfica.
    fig.update_layout(template='plotly_white')
    fig.show()
    
# Genera el dataframe con los datos necesarios para hacer las gráficas.
graph_iy = pd.DataFrame({'x':iy[0:12,0].reshape((12)),'y':iy_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_pf = pd.DataFrame({'x':pf[0:12,1].reshape((12)),'y':pf_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_e = pd.DataFrame({'x':e[0:12,2].reshape((12)),'y':e_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)
graph_dif = pd.DataFrame({'x':dif[0:12,3].reshape((12)),'y':dif_pred.reshape((12))}, columns=['x','y']).sort_values(by=['x'], ascending= True)

# Se crean las gráficas.
cp_plot(graph_iy, 'Iy')
cp_plot(graph_pf, 'PF')
cp_plot(graph_e, 'e')
cp_plot(graph_dif, 'dIf')

1/1 [==============================] - 0s 16ms/step


In [ ]:
model.save('Modelos_regresion/modelo1')